In [1]:
! pip3 install -qU langchain-upstage pypdf

In [2]:

%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [14]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("solar_sample.pdf")
docs = loader.load()  # or layzer.lazy_load()
print(docs[0].page_content[:1000])



Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)


Model Size Type H6 (Avg.) ARC HellaSwag MMLU TruthfulQA Winogrande GSM8KSOLAR 10.7B-Instruct⇠11BAlignment-tuned74.2071.0888.1666.2171.4383.5864.75Qwen 72B⇠72B Pretrained 73.60 65.19 85.9477.3760.19 82.4870.43Mixtral 8x7B-Instruct-v0.1⇠47B Instruction-tuned 72.62 70.22 87.63 71.16 64.58 81.37 60.73Yi 34B-200K⇠34B Pretrained 70.81 65.36 85.58 76.06 53.64 82.56 61.64Yi 34B⇠34B Pretrained 69.42 64.59 85.69 76.35 56.23 83.03 50.64Mixtral 8x7B-v0.1⇠47B Pretrained 68.42 66.04 86.49 71.82 46.78 81.93 57.47Llama 2 70B⇠70B Pretrained 67.87 67.32 87.33 69.83 44.92 83.74 54.06Falcon 180B⇠180B Pretrained 67.85 69.4588.8670.50 45.4786.9045.94SOLAR 10.7B⇠11BPretrained66.0461.9584.6065.4845.0483.6655.50Qwen 14B⇠14B Pretrained 65.86 58.28 83.99 67.70 49.43 76.80 58.98Mistral 7B-Instruct-v0.2⇠7B Instruction-tuned 65.71 63.14 84.88 60.78 68.26 77.19 40.03Yi 34B-Chat⇠34B Instruction-tuned 65.32 65.44 84.16 74.90 55.37 80.11 31.92Mistral 7B⇠7B Pretrained 60.97 59.98 83.31 64.16 42.15 78.37 37.83Table 2: Ev

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context. 
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [15]:
chain.invoke({"question": "What is the license under which SOLAR 10.7B is available?", "Context": docs})

'The information is not present in the context.'

In [8]:
chain.invoke({"question": "Who is the created SOLAR 10.7B?", "Context": context})

'The information is not present in the context.'

In [9]:
chain.invoke({"question": "Did Google provide resources for the SOLAR 10.7B project?", "Context": context})

'The information is not present in the context.'

In [10]:
chain.invoke({"question": "Why DUS is good", "Context": context})

'DUS is good because it is a simple yet effective method for scaling up high-performance LLMs from small ones, as demonstrated by the superior performance of SOLAR 10.7B in various NLP tasks.'

In [11]:
answer = chain.invoke({"question": "What is the name of the variant fine-tuned for instruction-following capabilities?", "Context": context})
print(answer)

The variant fine-tuned for instruction-following capabilities is called SOLAR 10.7B-Instruct.


## Groundedness Check with LangChain and Upstage
![Groundedness](./figures/gc.png)


In [12]:
# GC
from langchain_upstage import UpstageGroundednessCheck

groundedness_check = UpstageGroundednessCheck()

answer = chain.invoke(
    {
        "question": "What is the name of the variant fine-tuned for instruction-following capabilities?",
        "Context": context,
    }
)
print(answer)
gc_result = groundedness_check.invoke({"context": context, "answer": answer})

print(gc_result)
if gc_result.lower().startswith("grounded"):
    print("✅ Groundedness check passed")
else:
    print("❌ Groundedness check failed")

The name of the variant fine-tuned for instruction-following capabilities is SOLAR 10.7B-Instruct.
grounded
✅ Groundedness check passed


In [13]:
# GC
from langchain_upstage import UpstageGroundednessCheck

groundedness_check = UpstageGroundednessCheck()

answer = "Solar 10.7B is available to the public with a non-commercial license."
gc_result = groundedness_check.invoke({"context": context, "answer": answer})

print(gc_result)
if gc_result.lower().startswith("grounded"):
    print("✅ Groundedness check passed")
else:
    print("❌ Groundedness check failed")

notGrounded
❌ Groundedness check failed


# Excercise 

Write code to check GC and make LLM more secure. For example, check GC several times until LLM gives you a reliable answer.